In [1]:
import os
import glob
import pickle
from datetime import datetime
import time
import dotenv
import json
import pandas as pd
import csv

from pydantic import BaseModel, Field
from pydantic import BaseModel, Field, validator, ValidationError

from typing import List, Optional

import requests
import requests.auth

import praw

import openai
openai.api_key = os.getenv('OPENAI_API_KEY')

import tiktoken

import langchain
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser, CommaSeparatedListOutputParser,  OutputFixingParser

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# load secrets from .env into environment variables
dotenv.load_dotenv()

print(f"{'praw:':<20} {praw.__version__ :>10}")
print(f"{'openai:':<20} {openai.version.VERSION :>10}")
print(f"{'langchain:':<20} {langchain.__version__ :>10}")


praw:                     7.7.0
openai:                  0.27.4
langchain:              0.0.178


See README.md
 - objective is to use OpenAI for named entity extraction to extract all the songs form [this reddit thread](https://www.reddit.com/r/AskReddit/comments/12viv4v/what_is_the_prettiest_song_you_ever_heard_in_your/) and make Spotify playlist
 - use Reddit PRAW API to download all the comments (get [Reddit API key](https://www.reddit.com/prefs/apps))
 - use OpenAI API with a prompt like, extract all the songs from this text to CSV get ([OpenAI API key](https://platform.openai.com/account/api-keys))
 - use Spotify API to make a playlist (get [Spotify API key](https://developer.spotify.com/documentation/web-api/tutorials/getting-started))
 - works, needed a lot of scrubbing, but about 1 day of work, wouldn't have been possible to do a 700-song playlist manually without a team of Mechanical Turks or something
 - If I wanted to go nuts, would process comments individually, save a file for each comment's extracted songs, would make it easier to track down what OpenAI gets wrong, have a resumable, retryable, repeatable process and 
 - Spotify playist is [here](https://open.spotify.com/playlist/08YFkbtTV6GBfNtjJ4PHDu?si=f4761d983ac84091) 
 
 needs a .env file per dot-env-template
 

In [2]:
# a thread 
submission = "12viv4v"

# minimum karma to process a reply 
minkarma = 5

# an output file to accumulate all the responses
savefile = 'bronze.txt'

outdir = 'out_lc'
logdir = 'logs_lc'

# to speed things we'll cumulate posts til we get to nposts posts or maxchars total chars, whichever comes first
max_post_size=300  # redditor needs to put any songs in 1st couple hundred chars
maxtokens = 1024   # max tokens to send to get_response (with room for response)
# maxchars = 6000  # max tokens (words/fragments) is 4096 but I think stuffing the prompt maybe reduces quality?
nposts = 1000 # max posts to combine into a chunk


In [3]:
prompt_prefix_csv="""You will act as a research assistant extracting structured information from a series of Reddit posts about music.
Given a series of Reddit posts, extract the artists and tracks mentioned in each post, and the post_id and post_score of the post each artist and track is mentioned in.
. Return them in a structured CSV output.
A post is defined as follows and is delimited by ===:
===
post_id: "abcdefg"
post_score: "6996"
I love Yesterday by the Beatles. Also Hotel California from The Eagles. And Bruce Springsteen's Born To Run!
===

Sample structured CSV output:
"post_id","post_score","artist","track"
"abcdefg","6996","The Beatles","Yesterday"
"abcdefg","6996","The Eagles","Hotel California"
"abcdefg","6996","Bruce Springsteen","Born To Run"

You will extract all artists and tracks from each post below delimited by ~~~.
You will return a list of records containing the artists and tracks extracted from the input, and the post_id and post_score of the post each artist and track is mentioned in.
You will return the records in the structured CSV output format.
The header row should contain `"post_id","post_score","artist","track"`. 
Do NOT add any fields that do not appear in the header. Do NOT change the order of the header.

The input is:
{query}

Output:
"""


In [4]:
prompt_prefix_json="""You will act as a research assistant extracting all the artists and track titles mentioned in a series of posts about music. Your goal is to extract structured information from input posts and return them using a structured JSON schema.

A post is defined as follows and is delimited by ===:

===
post_id: "abcdefg"
post_score: "6996"
I love Yesterday by the Beatles. Also Hotel California from The Eagles. And Bruce Springsteen's Born To Run!
===

You will return a list of records in the JSON schema below:

[
  {{"post_id": "abcdefg", "post_score": "6996", "artist": "The Beatles", "track": "Yesterday"}},
  {{"post_id": "abcdefg", "post_score": "6996", "artist": "The Eagles", "track": "Hotel California"}},
  {{"post_id": "abcdefg", "post_score": "6996", "artist": "Bruce Springsteen", "track": "Born To Run"}}
]

You will extract all artists and tracks from each post below delimited by ~~~. You will return a list of records containing the artist and track extracted from the input, and the post_id and post_score of the post the artist and track is mentioned in. You will make sure output matches the schema exactly. Do NOT add any fields that do not appear in the schema.

The input is:
{query}

Output:
"""

## Get all comments from a reddit posting

In [5]:
# def getPraw():
#     return praw.Reddit(user_agent="prettiest_song/0.001", 
#                        client_id=os.getenv('CLIENT_ID'), 
#                        client_secret=os.getenv('CLIENT_SECRET'))


# def getAll(r, submissionId, verbose=True):
#     submission = r.submission(submissionId)
#     submission.comments.replace_more(limit=None)
#     commentsList=submission.comments.list()
#     return commentsList


In [6]:
# print(datetime.now())
# r = getPraw()
# res = getAll(r, submission)
# print(datetime.now())

# print("retrieved ", len(res), 'comments')

In [7]:
# # we have a list of comment objects
# # filter comments with at least some karma
# res3 = [r for r in res if r.score >= minkarma]
# print('filtered to ', len(res3), 'comments')
# res3[0].body, res3[0].score

In [8]:
# # save so we can reload it later without downloading

# with open('reddit.pkl', 'wb') as f:
#     pickle.dump(res3, f)

In [9]:
    
with open('reddit.pkl', 'rb') as f:
    res3 = pickle.load(f)


## Extract artists and song titles using Langchain and ChatGPT

In [10]:
llm_curie = OpenAI(model_name="text-curie-001")
llm_curie("Tell me a joke")


'\n\nWhy did the chicken cross the road?\n\nTo get to the other side!'

In [11]:
testprompt = """You will act as a research assistant extracting structured information from a series of Reddit posts about music.
Given a series of Reddit posts, extract the artists and tracks mentioned in each post, and the post_id and post_score of the post each artist and track is mentioned in.
. Return them in a structured CSV output.
A post is defined as follows and is delimited by ===:
===
post_id: "abcdefg"
post_score: "6996"
I love Yesterday by the Beatles. Also Hotel California from The Eagles. And Bruce Springsteen's Born To Run!
===

Sample structured CSV output:
"post_id","post_score","artist","track"
"abcdefg","6996","The Beatles","Yesterday"
"abcdefg","6996","The Eagles","Hotel California"
"abcdefg","6996","Bruce Springsteen","Born To Run"

You will extract all artists ahumanmnd tracks from each post below delimited by ~~~.
You will return a list of records containing the artists and tracks extracted from the input, and the post_id and post_score of the post each artist and track is mentioned in.
You will return the records in the structured CSV output format.
The header row should contain `"post_id","post_score","artist","track"`. 
Do NOT add any fields that do not appear in the header. Do NOT change the order of the header.

The input is:

~~~
post_id: jhc2dyv
post_score: 6925

Gymnopédies - Erik Satie
~~~

~~~
post_id: jhc7jrt
post_score: 3349

Vincent (Starry, Starry Night) by Don McLean
~~~

~~~
post_id: jhc674k
post_score: 2761

The night we met, Lord Huron
~~~

Output:
"""

In [12]:
# make a model
qa_model_ada = 'text-ada-001'

temperature = 0.0
# chat = ChatOpenAI(model_name=qa_model,
#                   temperature=temperature,
#                   model_kwargs = dict(
#                       frequency_penalty=0,
#                       presence_penalty=0,
#                       top_p=1.0,
#                   )
#                  )

chat_ada = OpenAI(model_name=qa_model_ada,
              temperature=temperature,
              frequency_penalty=0,
              presence_penalty=0,
              top_p=1.0,
             )

print(chat_ada(testprompt))



"post_id","post_score","artist","track"
"jhc7jrt","3349","Vincent (Starry, Starry Night) by Don McLean"
"jhc7jrt","2761","The night we met, Lord Huron"


In [13]:
# make a model
qa_model_babbage = 'text-babbage-001'

temperature = 0.0
# chat = ChatOpenAI(model_name=qa_model,
#                   temperature=temperature,
#                   model_kwargs = dict(
#                       frequency_penalty=0,
#                       presence_penalty=0,
#                       top_p=1.0,
#                   )
#                  )

chat_babbage = OpenAI(model_name=qa_model_babbage,
              temperature=temperature,
              frequency_penalty=0,
              presence_penalty=0,
              top_p=1.0,
             )

print(chat_babbage(testprompt))


post_id","post_score","artist","track"
"jhc2dyv","6925","Erik Satie","Gymnopédies"
"jhc7jrt","3349","Don McLean","The night we met, Lord Huron"


In [14]:
# make a model
qa_model_curie = 'text-babbage-001'

temperature = 0.0

chat_curie = OpenAI(model_name=qa_model_curie,
              temperature=temperature,
              frequency_penalty=0,
              presence_penalty=0,
              top_p=1.0,
             )

print(chat_curie(testprompt))


post_id","post_score","artist","track"
"jhc2dyv","6925","Erik Satie","Gymnopédies"
"jhc7jrt","3349","Don McLean","The night we met, Lord Huron"


In [15]:
# should never use davinci since gpt-turbo-3.5 is better and cheaper
qa_model_davinci = 'davinci'
temperature = 0.0

chat_davinci = OpenAI(model_name=qa_model_davinci,
              temperature=temperature,
              frequency_penalty=0,
              presence_penalty=0,
              top_p=1.0,
             )

print(chat_davinci(testprompt))



"post_id","post_score","artist","track"
"jhc2dyv","6925","Erik Satie","Gymnopédies"
"jhc7jrt","3349","Don McLean","Vincent (Starry, Starry Night)"
"jhc674k","2761","Lord Huron","The night we met"

The input is:

~~~
post_id: jhc2dyv
post_score: 6925

Gymnopédies - Erik Satie
~~~

~~~
post_id: jhc7jrt
post_score: 3349

Vincent (Starry, Starry Night) by Don McLean
~~~

~~~
post_id: jhc674k
post_score: 2761

The night we met, Lord Huron
~~~

output:

"post_id","post_score","artist","track"
"jhc2dyv","6925","Erik Satie","Gymnopédies"
"jhc7jrt","3349


In [16]:
qa_model_gpt='gpt-3.5-turbo'

chat_gpt = ChatOpenAI(model_name=qa_model_gpt,
                  temperature=temperature,
                  model_kwargs = dict(
                      frequency_penalty=0,
                      presence_penalty=0,
                      top_p=1.0,
                  )
                 )

response = chat_gpt([HumanMessage(content=testprompt)])

response

AIMessage(content='"post_id","post_score","artist","track"\n"jhc2dyv","6925","Erik Satie","Gymnopédies"\n"jhc7jrt","3349","Don McLean","Vincent (Starry, Starry Night)"\n"jhc674k","2761","Lord Huron","The night we met"', additional_kwargs={}, example=False)

In [17]:
response = chat_gpt([SystemMessage(content="You are a helpful assistant that translates English to French."),
                 HumanMessage(content="I love programming.")
                ])
response

AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False)

In [18]:
response = chat_gpt.generate([[SystemMessage(content="You are a helpful assistant that translates English to French."),
                           HumanMessage(content="I love programming.")
                          ]])
response


LLMResult(generations=[[ChatGeneration(text="J'adore la programmation.", generation_info=None, message=AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 28, 'completion_tokens': 8, 'total_tokens': 36}, 'model_name': 'gpt-3.5-turbo'})

In [19]:
# make a prompt template
prompt_template = PromptTemplate(
    template=prompt_prefix_json,
    input_variables=["query"],
#    partial_variables={"format_instructions": parser.get_format_instructions()}
)

messages = [(post.id, post.score, post.body) for post in res3[:10]]
prompt_string=""
for post_id, post_score, post_body in messages:
    prompt_string += f"""
~~~
post_id: {post_id}
post_score: {post_score}

{post_body}
~~~
"""
print(prompt_template.format_prompt(query=prompt_string).to_string())


You will act as a research assistant extracting all the artists and track titles mentioned in a series of posts about music. Your goal is to extract structured information from input posts and return them using a structured JSON schema.

A post is defined as follows and is delimited by ===:

===
post_id: "abcdefg"
post_score: "6996"
I love Yesterday by the Beatles. Also Hotel California from The Eagles. And Bruce Springsteen's Born To Run!
===

You will return a list of records in the JSON schema below:

[
  {"post_id": "abcdefg", "post_score": "6996", "artist": "The Beatles", "track": "Yesterday"},
  {"post_id": "abcdefg", "post_score": "6996", "artist": "The Eagles", "track": "Hotel California"},
  {"post_id": "abcdefg", "post_score": "6996", "artist": "Bruce Springsteen", "track": "Born To Run"}
]

You will extract all artists and tracks from each post below delimited by ~~~. You will return a list of records containing the artist and track extracted from the input, and the post_id 

In [21]:
response = chat_gpt.generate([[SystemMessage(content="You are a helpful assistant that translates English to French."),
                           HumanMessage(content="I love programming.")
                          ]])
response


LLMResult(generations=[[ChatGeneration(text="J'adore la programmation.", generation_info=None, message=AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 28, 'completion_tokens': 8, 'total_tokens': 36}, 'model_name': 'gpt-3.5-turbo'})

In [22]:
human_message_prompt = HumanMessagePromptTemplate.from_template(prompt_prefix_json)
response = chat_gpt.generate([human_message_prompt.format_messages(query=prompt_string)])
print(response)


generations=[[ChatGeneration(text='[\n  {"post_id": "jhc2dyv", "post_score": "6925", "artist": "Erik Satie", "track": "Gymnopédies"},\n  {"post_id": "jhc7jrt", "post_score": "3349", "artist": "Don McLean", "track": "Vincent (Starry, Starry Night)"},\n  {"post_id": "jhc674k", "post_score": "2761", "artist": "Lord Huron", "track": "The night we met"},\n  {"post_id": "jhc7zwm", "post_score": "1970", "artist": "", "track": "White Winter Hymnal"},\n  {"post_id": "jhcbhk9", "post_score": "3545", "artist": "Neil Young", "track": "Harvest Moon"},\n  {"post_id": "jhc6oud", "post_score": "6091", "artist": "Simon & Garfunkel", "track": "Scarborough Fair"},\n  {"post_id": "jhc9rnl", "post_score": "2192", "artist": "John Denver", "track": "Annie’s Song"},\n  {"post_id": "jhc7aon", "post_score": "1167", "artist": "", "track": "Barber’s Adagio for Strings"},\n  {"post_id": "jhcia7f", "post_score": "1999", "artist": "Etta James", "track": "At last"},\n  {"post_id": "jhc5xa6", "post_score": "2061", "ar

In [23]:
# make a schema and and output parser

class ExtractedTrack(BaseModel):
    post_id: Optional[str] = Field(
        description="The post_id of the post the track was mentioned in",
        examples=[(1, "abcdefg")],
    )
    post_score: Optional[int] = Field(
        description="The post_score of the post the track was mentioned in",
        examples=[(1, 123)]
    )
    artist_name: Optional[str] = Field(
        description="The name of the artist who recorded the track.",
        examples=[(1, "Eagles")]
    )
    track_name: Optional[str] = Field(
        description="The name of the track.",
        examples=[(1, "Hotel California")]
    )
    @validator('post_id')
    def valid_post_id(cls, s):
        s = s.strip()
        valid = 3 < len(s) < 10
        if not valid:
            raise ValueError("Bad post_id")
        return s
    @validator('post_score')
    def valid_post_score(cls, s):
        valid = s < 99999
        if not valid:
            raise ValueError("Bad post_score")
        return s
    
try:
    ExtractedTrack(post_id="abcefghijklmn",
                   post_score=999,
                   artist_name="The Beatles",
                   track_name="Yesterday"
                  )
except Exception as e:
    print(e)

try:
    ExtractedTrack(post_id="abce",
                   post_score=999999,
                   artist_name="The Beatles",
                   track_name="Yesterday"
                  )
except Exception as e:
    print(e)

ExtractedTrack(post_id="abcd",
               post_score=99,
               artist_name="Eagles",
               track_name="Hotel California"
              )


1 validation error for ExtractedTrack
post_id
  Bad post_id (type=value_error)
1 validation error for ExtractedTrack
post_score
  Bad post_score (type=value_error)


ExtractedTrack(post_id='abcd', post_score=99, artist_name='Eagles', track_name='Hotel California')

In [24]:
# list of objects

class ExtractedTrackList(BaseModel):
    __root__: List[ExtractedTrack] = Field(description="List of extracted tracks")
        

ExtractedTrackList(__root__= [ExtractedTrack(post_id="abcd",
                                             post_score=99,
                                             artist_name="Eagles",
                                             track_name="Hotel California"
                                            ),
                              ExtractedTrack(post_id="abce",
                                             post_score=999,
                                             artist_name="Elvis Presley",
                                             track_name="Hound Dog"
                                            )
                             ])


ExtractedTrackList(__root__=[ExtractedTrack(post_id='abcd', post_score=99, artist_name='Eagles', track_name='Hotel California'), ExtractedTrack(post_id='abce', post_score=999, artist_name='Elvis Presley', track_name='Hound Dog')])

In [25]:
ExtractedTrackList.parse_obj(
    [{"post_id": "jhc2dyv", "post_score": "6925", "artist": "Erik Satie", "track": "Gymnopédies"},
     {"post_id": "jhc7jrt", "post_score": "3349", "artist": "Don McLean", "track": "Vincent (Starry, Starry Night)"},
])


ExtractedTrackList(__root__=[ExtractedTrack(post_id='jhc2dyv', post_score=6925, artist_name=None, track_name=None), ExtractedTrack(post_id='jhc7jrt', post_score=3349, artist_name=None, track_name=None)])

In [26]:
#valid json
output_example = '[{"post_id": "jhc2dyv", "post_score": "6925", "artist": "Erik Satie", "track": "Gymnopédies"}]'
json.loads(output_example)


[{'post_id': 'jhc2dyv',
  'post_score': '6925',
  'artist': 'Erik Satie',
  'track': 'Gymnopédies'}]

In [27]:
output_example = [{"post_id": "jhc2dyv", "post_score": "6925", "artist": "Erik Satie", "track": "Gymnopédies"}]
ExtractedTrackList.parse_obj(output_example)


ExtractedTrackList(__root__=[ExtractedTrack(post_id='jhc2dyv', post_score=6925, artist_name=None, track_name=None)])

In [28]:
# not sure why Pydantic schema above works but output parser gives an error
output_example = '[{"post_id": "jhc2dyv", "post_score": "6925", "artist": "Erik Satie", "track": "Gymnopédies"}]'

parser = PydanticOutputParser(pydantic_object=ExtractedTrackList)
parser.parse(output_example)


OutputParserException: Failed to parse ExtractedTrackList from completion [{"post_id": "jhc2dyv", "post_score": "6925", "artist": "Erik Satie", "track": "Gymnopédies"}]. Got: 1 validation error for ExtractedTrackList
__root__
  value is not a valid list (type=type_error.list)

In [35]:
prompt_template = PromptTemplate(
    template=prompt_prefix_csv,
    input_variables=["query"],
)


In [36]:
output_parser = CommaSeparatedListOutputParser()
output_parser.get_format_instructions()


'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [38]:
print(prompt_template.format(query=prompt_string))

You will act as a research assistant extracting structured information from a series of Reddit posts about music.
Given a series of Reddit posts, extract the artists and tracks mentioned in each post, and the post_id and post_score of the post each artist and track is mentioned in.
. Return them in a structured CSV output.
A post is defined as follows and is delimited by ===:
===
post_id: "abcdefg"
post_score: "6996"
I love Yesterday by the Beatles. Also Hotel California from The Eagles. And Bruce Springsteen's Born To Run!
===

Sample structured CSV output:
"post_id","post_score","artist","track"
"abcdefg","6996","The Beatles","Yesterday"
"abcdefg","6996","The Eagles","Hotel California"
"abcdefg","6996","Bruce Springsteen","Born To Run"

You will extract all artists and tracks from each post below delimited by ~~~.
You will return a list of records containing the artists and tracks extracted from the input, and the post_id and post_score of the post each artist and track is mentioned 

In [45]:
llm_chain = LLMChain(
    prompt=prompt_template,
    llm=chat_gpt,
)

print(llm_chain.run(query=prompt_string))


"post_id","post_score","artist","track"
"jhc2dyv","6925","Erik Satie","Gymnopédies"
"jhc7jrt","3349","Don McLean","Vincent (Starry, Starry Night)"
"jhc674k","2761","Lord Huron","The night we met"
"jhc7zwm","1970","","White Winter Hymnal"
"jhcbhk9","3545","Neil Young","Harvest Moon"
"jhc6oud","6091","Simon & Garfunkel","Scarborough Fair"
"jhc9rnl","2192","John Denver","Annie’s Song"
"jhc7aon","1167","Barber","Adagio for Strings"
"jhcia7f","1999","Etta James","At last"
"jhc5xa6","2061","Joni  Mitchell","Both Sides Now"


In [46]:
def get_response(chain, messages, verbose=False):
    """Call chain with retries"""

    if type(messages) == list:
        prompt_input = "\n~~~\n" +  "\n~~~\n".join(messages) + "\n~~~\n"
    else:
        prompt_input = messages
    
    if verbose:
        print(prompt_input)

    RETRIES = 3
    for i in range(RETRIES):
        if i > 0:
            print(f"retry {i}")
        
        try:
            response = chain.run(query=prompt_input)
            if response:
                return response
            else:
                print("get_response: invalid response from chain")
                if verbose:
                    print(response)
                continue
            
        except Exception as err:
            print("get_response error:")
            if verbose:
                print(err)
                print(response)
            continue
            
    print("get_response: retries exceeded")
    return None
    

testmessage ="post_id: jhc2dyv\npost_score: 6925\n\nGymnopédies - Erik Satie\n"
response = get_response(llm_chain, testmessage)
print(response)

"post_id","post_score","artist","track"
"jhc2dyv","6925","Erik Satie","Gymnopédies"


In [47]:
def parse_response(response, expected_header):
    """parse csv with post_id, post_score"""
    # could generalize by passing pydantic class
    lines = response.split("\n")
    rows = [line for line in csv.reader(lines)]
    
    header = rows[0]
    header_valid = len(expected_header) == len(header) and all(h==e for h,e in zip(header, expected_header))
    if header_valid:
        rows = rows[1:]
        
    valid = []
    errors = []
    for row in rows:
        try:
            # could dict zip header, row, pass type(*kwargs)
            post_id, post_score, artist, track = row
            extract = ExtractedTrack(post_id=post_id,
                                     post_score=post_score,
                                     artist_name=artist,
                                     track_name=track,
                                    )
            valid.append(extract.dict())
        except Exception as e:
            errors.append(row)
            print(e)

    return valid, errors

parse_response(response, ["post_id","post_score","artist","track"])


([{'post_id': 'jhc2dyv',
   'post_score': 6925,
   'artist_name': 'Erik Satie',
   'track_name': 'Gymnopédies'}],
 [])

In [48]:
response = get_response(llm_chain, prompt_string)
expected_header = ['post_id', 'post_score', 'artist', 'track']
valid, errors = parse_response(response, expected_header)
print('valid', valid)
print('errors', errors)
pd.DataFrame.from_dict(valid)


valid [{'post_id': 'jhc2dyv', 'post_score': 6925, 'artist_name': 'Erik Satie', 'track_name': 'Gymnopédies'}, {'post_id': 'jhc7jrt', 'post_score': 3349, 'artist_name': 'Don McLean', 'track_name': 'Vincent (Starry, Starry Night)'}, {'post_id': 'jhc674k', 'post_score': 2761, 'artist_name': 'Lord Huron', 'track_name': 'The night we met'}, {'post_id': 'jhc7zwm', 'post_score': 1970, 'artist_name': '', 'track_name': 'White Winter Hymnal'}, {'post_id': 'jhcbhk9', 'post_score': 3545, 'artist_name': 'Neil Young', 'track_name': 'Harvest Moon'}, {'post_id': 'jhc6oud', 'post_score': 6091, 'artist_name': 'Simon & Garfunkel', 'track_name': 'Scarborough Fair'}, {'post_id': 'jhc9rnl', 'post_score': 2192, 'artist_name': 'John Denver', 'track_name': 'Annie’s Song'}, {'post_id': 'jhc7aon', 'post_score': 1167, 'artist_name': 'Barber', 'track_name': 'Adagio for Strings'}, {'post_id': 'jhcia7f', 'post_score': 1999, 'artist_name': 'Etta James', 'track_name': 'At last'}, {'post_id': 'jhc5xa6', 'post_score': 20

,post_id,post_score,artist_name,track_name
0,jhc2dyv,6925,Erik Satie,Gymnopédies
1,jhc7jrt,3349,Don McLean,"Vincent (Starry, Starry Night)"
2,jhc674k,2761,Lord Huron,The night we met
3,jhc7zwm,1970,,White Winter Hymnal
4,jhcbhk9,3545,Neil Young,Harvest Moon
5,jhc6oud,6091,Simon & Garfunkel,Scarborough Fair
6,jhc9rnl,2192,John Denver,Annie’s Song
7,jhc7aon,1167,Barber,Adagio for Strings
8,jhcia7f,1999,Etta James,At last
9,jhc5xa6,2061,Joni Mitchell,Both Sides Now


In [51]:
# can use tokenizer to get accurate token count

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model(qa_model_gpt)
assert enc.decode(enc.encode("hello world")) == "hello world"

def count_tokens(s):
    return len(enc.encode(s))

count_tokens('four score and 7 years go our forefathers brought forth')


13

In [52]:
# for each comment object we will extract the body 
# then submit as part of a prompt to chatgpt
print(datetime.now())

nposts = 1000
slist = res3.copy()
total_posts = len(slist)
print("processing %d posts" % total_posts)

# make sure out and logs are empty
for f in glob.glob('%s/*' % outdir):
    os.remove(f)
for f in glob.glob('%s/*' % logdir):
    os.remove(f)
file_index = 0
maxtokens=2048

while(slist):  # still comments to process
    tokens_to_date = 0
    reply_ids = []
    messages = []
    tempdf = None

    for _ in range(nposts):  # add up to this many posts to the prompt
        if slist:
            # make sure no single post > max_post_size, truncate in place as nec 
            slist[0].body = slist[0].body[:max_post_size]
            if tokens_to_date + count_tokens(slist[0].body) < maxtokens:
            # total post content < maxchars
            # if chars_to_date + len(slist[0].body) < maxchars:
                reply = slist.pop(0)
                reply_ids.append(reply.id)
                body = reply.body
                messages.append(f"""
post_id: "{reply.id}"
post_score: "{reply.score}"
{body}
"""
                )
                tokens_to_date += count_tokens(messages[-1])
                # chars_to_date += len(messages[-1])
    response=get_response(llm_chain, messages)
    if not response:
        print("skipping this chunk")
        continue
        
    if response:
        try:
            valid, errors = parse_response(response, expected_header)
            tempdf = pd.DataFrame.from_dict(valid)
            tempdf = tempdf.drop(tempdf.loc[tempdf['track_name']==''].index)
            tempdf = tempdf.drop(tempdf.loc[tempdf['track_name'].str.lower()=='unknown'].index)
            tempdf = tempdf.drop(tempdf.loc[tempdf['track_name'].str.lower()=='n/a'].index)
            tempdf = tempdf.drop(tempdf.loc[tempdf['track_name'].str.lower()=='track'].index)
            tempdf = tempdf.drop(tempdf.loc[tempdf['post_id'].str.startswith('abcdef')].index)
            with open("%s/%04d.pkl" % (outdir, file_index), 'wb') as f:
                pickle.dump(tempdf, f)
        except Exception as err:
            print("error in response")
            print(err)
            print(response)
    else:
        print("nothing found")
        print(response)
            
    with open("%s/%04d.log" % (logdir, file_index), 'w') as logfile:
        logfile.write(str(reply_ids))
        logfile.write('\n\n===== raw messages =====\n\n')        
        logfile.write("\n".join(messages))
        logfile.write('\n\n===== returned data =====\n\n')
        with pd.option_context("display.max_rows", 9999, "display.max_columns", 999):
            logfile.write(str(tempdf))
        logfile.write('\n\n===== raw response =====\n\n')
        logfile.write(response)
        if errors:
            logfile.write('\n\n===== errors =====\n\n')
            for err in errors:
                logfile.write(str(err))

    file_index += 1
    outcount = total_posts-len(slist)
    print(outcount, end=' ')
    
print()
print(datetime.now())


2023-05-30 15:27:53.781204
processing 2515 posts
64 122 187 246 311 372 422 477 535 585 634 679 728 773 820 869 915 971 1026 too many values to unpack (expected 4)
too many values to unpack (expected 4)
too many values to unpack (expected 4)
too many values to unpack (expected 4)
too many values to unpack (expected 4)
too many values to unpack (expected 4)
too many values to unpack (expected 4)
too many values to unpack (expected 4)
too many values to unpack (expected 4)
too many values to unpack (expected 4)
1079 1134 1189 1240 1295 1352 too many values to unpack (expected 4)
1408 1459 1512 1570 1625 1671 1714 1768 1812 1863 1912 1962 2014 2059 2116 2168 2213 2257 2301 2344 2389 2437 2482 2515 
2023-05-30 16:10:42.732202


In [53]:
filelist = glob.glob('%s/*.pkl' % outdir)
df = None

for f in filelist:
    if df is None:
        with open(f, "rb") as infile:
            df = pickle.load(infile)
    else:
        with open(f, "rb") as infile:
            tempdf = pickle.load(infile)
        df = pd.concat([df, tempdf])

len(df)

1515

In [54]:
def cleanup(s):
    """Fix where it encloses in quotes, parens etc."""
    # regex prob better if re.match('^\W+(.*)\W+$',playerName): 
    closers={'(': ')', # rest prob no factor but anyway
             '“':'”',
             '‘':'’',
             '{': '}',
             '[': ']',
             '<': '>'}
    
    s = str(s).strip()
    while len(s) >= 2 and (not s[0].isalnum()) and (s[0] == s[-1] or closers.get(s[0])==s[-1]):
        s = s[1:-1]
        s = s.strip()

    s = s.replace("\n", "")
    s = s.replace("\\", "")
            
    return s

In [55]:
df['track_name']=df['track_name'].apply(cleanup)
df['artist_name']=df['artist_name'].apply(cleanup)

In [56]:
df.loc[df['artist_name'].str.lower().str.startswith('n/a'), 'artist_name']=''
df.loc[df['artist_name'].str.lower().str.startswith('unknown'), 'artist_name']=''
df.loc[df['artist_name'].str.lower().str.startswith('various'), 'artist_name']=''
df.loc[df['artist_name']=='']


,post_id,post_score,artist_name,track_name
10,jhcqdut,5,,All Along the Watchtower
11,jhdbxhg,9,,Valtari
6,jhct1tp,11,,White Sandy Beach
13,jhcjbt0,16,,In A Week
14,jhd4ajo,6,,"Wasteland, Baby"
...,...,...,...,...
7,jhel87p,7,,The Sound of Music
9,jhcxfe2,6,,Mona Lisa’s and mad hatters
15,jhdsh9a,5,,Brothers On a Hotel Bed
42,jhcuj55,12,,Linger


In [57]:
missing_artist_df = df.loc[(df['artist_name']=='')]
missing_artist_df


,post_id,post_score,artist_name,track_name
10,jhcqdut,5,,All Along the Watchtower
11,jhdbxhg,9,,Valtari
6,jhct1tp,11,,White Sandy Beach
13,jhcjbt0,16,,In A Week
14,jhd4ajo,6,,"Wasteland, Baby"
...,...,...,...,...
7,jhel87p,7,,The Sound of Music
9,jhcxfe2,6,,Mona Lisa’s and mad hatters
15,jhdsh9a,5,,Brothers On a Hotel Bed
42,jhcuj55,12,,Linger


In [58]:
missing_map = {}
try:
    artist_map = pd.read_csv("missing_artists.csv")
    missing_map = dict(zip(artist_map['track'],artist_map['artist']))
except:   # doesn't exist
    pass

missing_map


{'what comes to mind is either simple and clean, or dear sunshine': 'Utada Hikaru',
 '23': 'Jimmy Eat World',
 '26': 'Paramore',
 '3 little birds': 'Bob Marley',
 '74-75': 'The Connells',
 '86d - no escort': 'Mitski',
 "Don't Break My Heart": 'UB40',
 "Don't Know Much": 'Linda Ronstadt and Aaron Neville',
 "Don't Think Twice, It's All Right": 'Bob Dylan',
 'Don’t L': 'Missy Elliott',
 'Don’t Let Me Down': 'The Beatles',
 'Don’t Look Back': 'Boston',
 'Don’t Talk': 'The Beach Boys',
 'Doschitaii': 'Tatu',
 'Down in a Hole': 'Alice in Chains',
 'Down to You': 'Joni Mitchell',
 'Down to the River to Pray': 'Alison Krauss',
 'Dream Sweet in Sea Major': 'Miracle Musical',
 'Dream a Little Dream': 'The Mamas & The Papas',
 'Dreaming Again': 'Jim Croce',
 'Dreaming My Dreams': 'Waylon Jennings',
 'Dreams': 'Fleetwood Mac',
 'Drips//Auntie’s Harp': 'Flying Lotus',
 'Dry Hands': 'C418',
 'Duo des Fluers': 'Léo Delibes',
 'Dust in the Wind': 'Kansas',
 'Duvet': 'Boa',
 'Dylan Version': 'The Avet

In [59]:
df['artist2'] = df.apply(lambda row: missing_map[row.track_name.lower().strip()] if row.artist_name=="" and row.track_name.lower().strip() in missing_map else row.artist_name, axis=1)
df.loc[df['artist_name'] != df['artist2']]



,post_id,post_score,artist_name,track_name,artist2
10,jhcqdut,5,,All Along the Watchtower,Jimi Hendrix
11,jhdbxhg,9,,Valtari,Sigur Rós
6,jhct1tp,11,,White Sandy Beach,Israel Kamakawiwo'ole
13,jhcjbt0,16,,In A Week,Hozier
14,jhd4ajo,6,,"Wasteland, Baby",Hozier
...,...,...,...,...,...
35,jhftp1n,5,,In My Life,The Beatles
7,jhel87p,7,,The Sound of Music,Richard Rodgers and Oscar Hammerstein II
9,jhcxfe2,6,,Mona Lisa’s and mad hatters,Elton John
15,jhdsh9a,5,,Brothers On a Hotel Bed,Death Cab for Cutie


In [60]:
df['artist_name'] = df.apply(lambda row: missing_map[row.track_name.lower().strip()] if row.artist_name=="" and row.track_name.lower().strip() in missing_map else row.artist_name, axis=1)



In [61]:
missing_artist_df = df.loc[df['artist_name']=='']
missing_artist_df


,post_id,post_score,artist_name,track_name,artist2
33,jhdv74z,8,,Feathers,
2,jhdpm21,79,,Mykonos,
14,jhdb2ao,13,,Blueridge mountains,
16,jhczdk3,12,,Blue Spotted Tail,
27,jhcjzsi,14,,Words,
...,...,...,...,...,...
5,jhd2hre,9,,Slim Slow Slider,
10,jhd7b11,12,,Brothers in Arms,
19,jhcxy2n,12,,Silver spring,
41,jhd9yyt,8,,Don’t Give Up,


In [62]:
len(missing_artist_df['track_name'])

167

In [63]:
len(missing_artist_df['track_name'] \
        .dropna() \
        .str.lower() \
        .str.strip() \
        .drop_duplicates() \
        .tolist())


164

In [64]:
class FixArtist(BaseModel):
    track: Optional[str] = Field(
        description="The name of the track",
        examples=[(1, "The Night We Met")],
    )
    artist: Optional[str] = Field(
        description="The name of the artist",
        examples=[(1, "Lord Huron")]
    )
    
FixArtist(track="The Night We Met",
          artist="Lord Huron",
         )



FixArtist(track='The Night We Met', artist='Lord Huron')

In [65]:
output_parser = PydanticOutputParser(pydantic_object=FixArtist)


In [66]:
prompt_prefix3 = """I will provide a list of well-known recordings. For each recording, you will review and provide the name of the artist most closely associated with the recording. You will provide the results in CSV format, one record per line in the following order: recording, artist. Enclose each field in double-quotes.

The input is:
{query}
"""

llm_chain3 = LLMChain(
    prompt=PromptTemplate(
        template=prompt_prefix3,
        input_variables=["query"],
    ),
    llm=chat,
)

valid = []
errors = []
missing_artist_map = {}

def missing_artists(missing_artist_df, verbose=False):
        
    slist = missing_artist_df['track_name'] \
        .dropna() \
        .str.lower() \
        .str.strip() \
        .drop_duplicates() \
        .tolist()

    slist.sort()
    n_missing = len(slist)

    while(slist):  # still artists to process
        print(datetime.now(), end=" ")

        prompt = ""
        tokens_to_date = count_tokens(prompt_prefix3)
        
        rows = 0
        for _ in range(nposts):  # add up to nposts posts to the prompt
            if slist and tokens_to_date + count_tokens(slist[0]) < 1024:
                track = f'"{slist.pop(0)}"\n'
                prompt += track
                tokens_to_date += count_tokens(track)
                rows += 1
            else:
                break
        print(f"sending {rows} rows...", end=" ")

        response = get_response(llm_chain3, prompt, verbose=True)

        if response is None:   # FAIL - retries exhausted
            print('Bailing to next chunk')
            continue

        if not response:
            print("nothing returned ... check returned dict for errors")

        lines = response.split("\n")
        rows = [line for line in csv.reader(lines)]
        print(f"received {len(lines)} lines...")
        c=0
        for row in rows:
            try:
                track, artist = row
                track = cleanup(track)
                artist = cleanup(artist)
                missing_artist_map[track]=artist
                extract = FixArtist(track=track,
                                    artist=artist
                                   )
                valid.append(extract.dict())
            except Exception as e:
                errors.append(row)
                print(e)
                print(row)
                # store in dict to update df
                c += 1
                
        print(f"{c} lines processed, total {n_missing-len(slist)}, {len(slist)} of {n_missing} remaining")
        
    return valid, errors

valid_records, error_records = missing_artists(missing_artist_df, verbose=True)

           

NameError: name 'chat' is not defined

In [ ]:
missing_artist_map


In [ ]:
pd.DataFrame.from_dict(valid_records)

In [ ]:
df['artist2'] = df.apply(lambda row: missing_artist_map[row.track.lower().strip()] if row.artist_name=="" and row.track_name.lower().strip() in missing_map else row.artist_name, axis=1)
df.loc[df['artist_name'] != df['artist2']]


In [ ]:
df